# **QnA Classification**

## 데이터 탐색

### 라이브러리 불러오기

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from wordcloud import WordCloud
import nltk
from nltk.tokenize import word_tokenize as nltk_word_tokenize
from nltk import FreqDist
from konlpy.tag import Okt, Mecab

import os

### 데이터 불러오기

In [2]:
path = "./train.csv"
law_data = pd.read_csv(path)
data = law_data.copy()

In [3]:
data.head()

,text,label
0,"self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, ...",코드2
1,현재 이미지를 여러개 업로드 하기 위해 자바스크립트로 동적으로 폼 여러개 생성하는데...,웹
2,glob.glob(PATH) 를 사용할 때 질문입니다.\n\nPATH에 [ ] 가 ...,코드2
3,"tmpp = tmp.groupby(by = 'Addr1', as_index=Fals...",코드2
4,filename = TEST_IMAGE + str(round(frame_sec)) ...,코드2


### 원본 데이터 확인

In [11]:
# 결측치 확인
data.isnull().sum()

text     0
label    0
dtype: int64

In [9]:
# 라벨 유형 별 갯수 및 비율
print(data["label"].value_counts())
print("-"*30)
label_percent = (data["label"].value_counts() / data["label"].count() * 100).round(2)
print(label_percent)

label
코드2       1097
웹          732
이론         728
시스템 운영     561
코드1        488
원격         100
Name: count, dtype: int64
------------------------------
label
코드2       29.60
웹         19.75
이론        19.64
시스템 운영    15.14
코드1       13.17
원격         2.70
Name: count, dtype: float64


### 텍스트 길이 확인

In [13]:
# 텍스트 길이 확인을 위한 컬럼 생성 및 값 부여
data["txt_len"] = data["text"].apply(len)

In [14]:
data.head()

,text,label,txt_len
0,"self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, ...",코드2,144
1,현재 이미지를 여러개 업로드 하기 위해 자바스크립트로 동적으로 폼 여러개 생성하는데...,웹,119
2,glob.glob(PATH) 를 사용할 때 질문입니다.\n\nPATH에 [ ] 가 ...,코드2,147
3,"tmpp = tmp.groupby(by = 'Addr1', as_index=Fals...",코드2,279
4,filename = TEST_IMAGE + str(round(frame_sec)) ...,코드2,144


In [16]:
# 텍스트 길이 최대값, 최소값, 평균 확인
data["txt_len"].max(), data["txt_len"].min(), data["txt_len"].mean()

(6012, 9, 200.95736643281165)

In [22]:
# 텍스트 길이 상위, 하위 값의 내용 형태 확인
data[data["txt_len"] > 5000].head()

,text,label,txt_len
472,NoSuchElementException Trac...,웹,5489
834,C:\dev\mysite_std&gt; python .\manage.py make...,웹,5187
1255,$ python manage.py createsuperuser\n\nYou have...,웹,5690
1617,교수님이 치시는거 따라 쳤는데 계속 저 오류가 반복합니다.\n\n# 대여소별 이용건...,코드2,5454
2561,sns.kdeplot(x = td[var])\nplt.show()\n\n------...,코드2,6012


In [21]:
data[data["txt_len"] < 20].head()

,text,label,txt_len
66,가치제안을 위한 필요사항,이론,13
226,타임아웃이 왜 뜨는지 궁금합니다!,웹,18
334,원격제어를 추가하는 것이 안됩니다,원격,18
336,특강은 녹화본 올라올까요?,시스템 운영,14
381,제출은 어떤 걸 하나요?,시스템 운영,13


### 명사 추출 및 품사 태깅

In [ ]:
from konlpy.tag import Okt, Mecab
from mecab import MeCab
import time

In [ ]:
okt = Okt()

# 명사 추출, 형태소 추출, 품사 태깅 진행
data["nouns"] = data["text"].apply(lambda x: okt.nouns(x))
data["morphs"] = data["text"].apply(lambda x: okt.morphs(x))
data["pos"] = data["text"].apply(lambda x: okt.pos(x))

In [ ]:
data.head()

### 불용어 처리

In [ ]:
# 불용어 사전을 이용하여 불용어 불러오기
with open("./Stopwords_ko.txt", "r", encoding="utf-8") as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

In [ ]:
# 불용어 처리
data["nouns_no_sw"] = data["nouns"].apply(lambda x: [word for word in x if word not in stopwords])

### 단어 빈도수 확인